yolov3은 총 3개의 scale이 출력된다. 이때 미리 정의되어있는 3개의 다른 anchor box를 가지고 각각의 scale에서 객체를 탐지한다.
3개의 scale x 서로다른 3개의 anchor box = 9 가 된다.
즉, anchor box를 총 9번 사용한다.

In [3]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans

anchors = [[(0.28, 0.22), (0.38, 0.48), (0.9, 0.78)]]

예시 값을 기준으로 볼 때 내가 탐지해야하는 객체는 총 4가지이다.<br>
때문에 나는 스케일 한개에 총 4개의 값이 있어햐 함.<br>
-> 밑에 함수를 수정해주어야 함.

1. if문으로 class별로 값을 구하기
2. 각 scale마다 값을 변경해서 구해야함.

In [51]:
def calculate_anchors_from_csv(csv_file, k, S):  # csv파일 불러와야하기 때문에 매개변수에 csv_file 써주기
    # Load the CSV file containing bounding box width and height
    df = pd.read_csv(csv_file)

    # Extract bounding box width and height from the DataFrame
    box_widths = df['w'].values
    box_heights = df['h'].values
    
    # Normalize bounding box width and height to the input_shape of the YOLO model
    norm_widths = box_widths / S[0]
    norm_heights = box_heights / S[0]
    print('input_shape : ', S[0])
    
    # Stack width and height into a single numpy array
    data = np.vstack((norm_widths, norm_heights)).T
    print('data : ', data)
    
    # Perform k-means clustering
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(data)

    # Get the centroids (anchors) from the k-means clustering
    anchors = kmeans.cluster_centers_
    print('anchors : ', anchors)
    
    # Denormalize the anchors back to the input_shape of the YOLO model
    anchors *= input_shape
    print('anchors *= input_shape : ', anchors)
    
    # Sort anchors in ascending order (optional)
    anchors = anchors[np.argsort(anchors[:, 0])]
    print('final : ', anchors)
    
    return anchors

In [52]:
# Example usage:
csv_file = './train/final.csv'  # Replace with the path to your CSV file
# 클러스터링은 데이터셋에 있는 bounding box들의 크기를 고려하여 적절한 anchor 값을 추정하는 데 도움이 됩니다.

# 한 객체를 탐지할 때 사용할 anchor box개수. 보통 5~9개로 지정한다.
# 이미지 특성에 맞게 값을 조정하기.
k = 5
# The input shape of your YOLO model (width, height)
S = [13, 26, 52]

anchors = calculate_anchors_from_csv(csv_file, k, S)
print("Calculated Anchors:", anchors)

input_shape :  (13, 13)
data :  [[3.23076923 8.30769231]]
anchors :  [[3.23076923 8.30769231]]
anchors *= input_shape :  [[ 42. 108.]]
final :  [[ 42. 108.]]
Calculated Anchors: [[ 42. 108.]]


C:\Users\1111\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\1111\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
